In [1]:
library(tidyverse)
employees <- read_csv("../_build/data/employee_data.csv")
employees$Salary <- parse_number(employees$Salary)
employees$Start_Date <- parse_date(employees$Start_Date, format = "%m/%d/%Y")
degreeLevels <- c("High School", "Associate's", "Bachelor's", "Master's", "Ph.D")
employees$Degree <- parse_factor(employees$Degree, levels = degreeLevels, ordered = TRUE)
offices <- read.csv("../_build/data/office.csv")
employees <- inner_join(employees, offices, by="ID")

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2


-- Attaching packages --------------------------------------- tidyverse 1.2.1 --


v ggplot2 3.1.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  


-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


Parsed with column specification:
cols(
  ID = col_double(),
  Name = col_character(),
  Gender = col_character(),
  Age = col_double(),
  Rating = col_double(),
  Degree = col_character(),
  Start_Date = col_character(),
  Retired = col_logical(),
  Division = col_character(),
  Salary = col_character()
)


# Summarising Data

Now that we know how to use the pipe, we can use it to quickly and efficiently summarise data. To start we first need to introduce the `summarise()` function from the `tidyverse`, which we can use to summarise one or more columns in a data frame. This function uses the following syntax:

```{admonition} Syntax
`tidyverse::summarise(df, summaryStat1 = ..., summaryStat2 = ..., ...)`
+ *Required arguments*
  - `df`: The data frame with the data. 
  - `summaryStat1 = ...`: The summary statistic we would like to calculate.
+ *Optional arguments*
  - `summaryStat2 = ..., ...`: Any additional summary statistics we would like to calculate.
```

For example, we can use `summarise()` to calculate all of the following at once from `employees`:

+ The average of `Salary`
+ The standard deviation of `Salary`
+ The minimum `Age`
+ The maximum `Age`

In [2]:
summarise(employees,  meanSalary = mean(Salary, na.rm = TRUE),
                          sdSalary = sd(Salary, na.rm = TRUE),
                          minAge = min(Age),
                          maxAge = max(Age))

meanSalary,sdSalary,minAge,maxAge
158034.3,39677.02,25,65


It is often useful to include the helper function `n()` within `summarise()`, which will calculate the number of observations in the data set. Note that this is similar to the `nrow()` function that we saw in the bootcamp, but `n()` only works within other `tidyverse` functions.

In [3]:
summarise(employees,  meanSalary = mean(Salary, na.rm = TRUE),
                          sdSalary = sd(Salary, na.rm = TRUE),
                          minAge = min(Age),
                          maxAge = max(Age),
                          nObs = n())

meanSalary,sdSalary,minAge,maxAge,nObs
158034.3,39677.02,25,65,908


The `summarise()` function is useful for calculating summary statistics, but it becomes even more powerful when we combine it with `group_by()`. 

```{admonition} Syntax
`tidyverse::group_by()`
```

Imagine that we wanted to calculate separate summary statistics for each of the three offices (`New York`, `Boston`, and `Detroit`) separately, not across the entire data set. To accomplish this, we can use the pipe to pass the data through `group_by()` first, then pass it through `summarise()`. Any variable(s) we specify in `group_by()` will be used to separate the data into distinct groups, and `summarise()` will be applied to each one of those groups separately. For example:

In [4]:
employees %>%

  group_by(office) %>%

  summarise(meanSalary = mean(Salary, na.rm=TRUE),
            sdSalary = sd(Salary, na.rm=TRUE),
            minAge = min(Age),
            maxAge = max(Age),
            nObs = n())

office,meanSalary,sdSalary,minAge,maxAge,nObs
Boston,157957.9,37388.57,25,65,294
Detroit,137587.2,38510.39,25,65,166
New York,165628.4,38978.08,25,65,448


From the output we can see that this calculate the summary statistics within each value of the `office` variable. We can also include more than one variable within `group_by()`. For example, imagine we wanted to calculate these summary statistics by gender within each office. All we would need to do is add `Gender` to the `group_by()`:

In [5]:
employees %>%

  group_by(office, Gender) %>%

  summarise(meanSalary = mean(Salary, na.rm=TRUE),
            sdSalary = sd(Salary, na.rm=TRUE),
            minAge = min(Age),
            maxAge = max(Age),
            nObs = n())

office,Gender,meanSalary,sdSalary,minAge,maxAge,nObs
Boston,Female,152778.1,34104.53,25,65,114
Boston,Male,161317.0,39106.56,25,65,180
Detroit,Female,133720.1,35552.39,25,65,69
Detroit,Male,140251.2,40401.41,25,64,97
New York,Female,160560.3,39787.98,25,65,220
New York,Male,170647.5,37584.82,25,65,228
